In [3]:
## Data scrapping prpm
from PyClasses.Scrapers.PRPMScraper import PRPMScraper
from PyClasses.Scrapers.PERIScraper import PeriScraper
PRPMscrap = PRPMScraper()

## find sinonim still buggy (will work if got time)
result = PRPMscrap.findWordMetaData('cari')
print(result)

#found smth can be preprocess if not done here, like there will have '\xa' character in the result string

{'cari': {'meanings': ['1. bp berusaha utk mendapat (menjumpai dll) sesuatu: di serata tempat saya ~ tapi tidak bertemu; 2. Mn harta milik (pencarian); ~ bagi harta pencarian yg dibahagi dua antara suami isteri (sewaktu bercerai); bercari Mn dicari (bkn persetujuan); kata dahulu bertepati, kata kemudian kata ~ prb janji mesti ditepati, hanya boleh diubah apabila sudah dirundingkan; bercari-cari dicari-cari, diada-adakan, di\xadbuat-buat: perkara yg ~; bercari-carian saling berusaha mencari; mencari 1. berusaha utk mendapat (mem\xadperoleh) sesuatu, melihat ke sana sini dan berusaha supaya menjumpai atau mendapat kembali sesuatu yg hilang atau tidak diketahui mana perginya; 2. mengada-adakan sesuatu, sengaja menimbulkan (mengadakan): dia memang suka ~ gaduh di tempat orang ramai; ~ akal (fikiran, jalan) berusaha mendapat cara utk mencapai tujuan; ~ bala = ~ susah membuat sesuatu yg akhirnya menyusahkan; ~ daya upaya = ~ ikhtiar berusaha sungguh\xad-sungguh; ~ enaknya (senangnya) sahaja 

In [1]:
from datetime import datetime, timedelta
from PyClasses.Scrapers.BHarianScraper import BHarianScraper
from pyspark.sql import SparkSession
from selenium import webdriver

def ArticleDataCollection(fromDate,categoryLink):
    spark = SparkSession\
        .builder\
        .appName("BHarian WebScraping")\
        .getOrCreate()

    ## crawl links
    bharianScraper = BHarianScraper()
    timeNow = datetime.now()
    crawlledUrls = bharianScraper.scrapArticleLinks(fromDate,categoryLink)
    url_RDD = spark.sparkContext.parallelize(crawlledUrls)
    if crawlledUrls is None:
        print('failed to get article links')
        return 'fail'
    
    ## parallelize process to scrap data
    scrappedData = url_RDD.map(BHarianScraper.scrapOneArticle).collect()
    for data in scrappedData:
        if data is None:
            scrappedData.remove(data)
            
    ## Save Data into HDFS
    df = spark.createDataFrame(scrappedData)
    df.write.format("parquet").mode("overwrite").save("DE-prj/RawData")
    timeEnd = datetime.now() 
    timeTaken = timeEnd - timeNow
    print(f'Processing Start Time: {timeNow}')
    print(f'Processing End Time: {timeEnd}')
    print(f'Total Time Taken: {timeTaken}')
    
    print('done')
    return 'success'

ArticleDataCollection(datetime(2024,12,9),'/berita/nasional')



24/12/10 18:54:16 WARN Utils: Your hostname, Liang. resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/12/10 18:54:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/12/10 18:54:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Article teaser found on page 0.
Current no. of link: 20
Article teaser found on page 1.
Current no. of link: 39
Article teaser found on page 2.
Current no. of link: 58
Article teaser found on page 3.
Current no. of link: 77
Article teaser found on page 4.
Current no. of link: 96
Article teaser found on page 5.
Current no. of link: 115
Article teaser found on page 6.
Current no. of link: 134
Article teaser found on page 7.
Current no. of link: 135


[Stage 0:===>                                                     (1 + 17) / 18]Timeout error on '/berita/nasional/2024/12/1334751/sistem-autogate-kod-qr-tergendala-akibat-kerosakan-suis-utama'. Scrap Attempt: 1 time(s)
Timeout error on '/berita/nasional/2024/12/1334631/media-prima-mampu-jadi-penyampai-maklumat-agenda-transformasi'. Scrap Attempt: 1 time(s)
[Stage 0:======>                                                  (2 + 16) / 18]Timeout error on '/berita/nasional/2024/12/1334670/siasatan-sprm-terhadap-hrd-corp-masih-berjalan-azam'. Scrap Attempt: 1 time(s)
[Stage 0:=========>                                               (3 + 15) / 18]Timeout error on '/berita/nasional/2024/12/1335049/perkeso-bayar-pampasan-hilang-upaya-rm212-bilion-tahun-lalu'. Scrap Attempt: 1 time(s)
Timeout error on '/berita/nasional/2024/12/1334954/wct-malls-perkenal-konsep-retailtainment-inovatif'. Scrap Attempt: 1 time(s)
[Stage 0:============>                                            (4 + 14) / 18]Time

Processing Start Time: 2024-12-10 18:54:18.405977
Processing End Time: 2024-12-10 19:01:08.251949
Total Time Taken: 0:06:49.845972
done


'success'

Row(title='Laporan siasatan kematian doktor pakar Hospital Lahad Datu diumum seawal bulan depan', authorDetails={'authName': 'Saadiah Ismail', 'relLink': '/authors/saadiah-ismail', 'email': 'Saadiah_ismail@bh.com.my'}, publishTime='Disember 10, 2024 @ 3:31pm', articleBrief='Menteri Kesihatan, Datuk Seri Dr Dzulkefly Ahmad. Gambar fail', contents=['KUALA LUMPUR: Kementerian Kesihatan (KKM) akan mengumumkan hasil siasatan jawatankuasa bebas yang ditubuhkan bagi menyiasat kematian pakar perubatan patologi di Hospital Lahad Datu, Sabah, seawal bulan depan.', 'Menteri Kesihatan, Datuk Seri Dr Dzulkefly Ahmad, berkata KKM menetapkan tempoh selama tiga bulan, namun jika hasil siasatan dapat diberikan dalam tempoh terdekat, pihaknya tidak akan menunggu sehingga Januari.', '"Kalau dapat lebih awal, kita akan mengadakan sidang media untuk memberikan butiran hasil siasatan memandangkan jawatankuasa bebas ini bukan di bawah KKM, jadi kita tidak boleh mendesak untuk menyegerakan siasatan.', '"Bagai

In [2]:
# for link in bharianScraper.relLinkToArticle_all:
#     print(link)
spark = SparkSession\
        .builder\
        .appName("ReadRawData")\
        .getOrCreate()
df_read = spark.read.parquet("DE-prj/RawData")
df_read.show()


df_filter = df_read.where(df_read['title'] == '')
df_filter.show()

print(df_read.count())


# Get the first 5 rows
rows = df_read.take(125)

# Print each row
print(rows[15])

24/12/10 19:01:17 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|               title|       authorDetails|         publishTime|        articleBrief|            contents|                 url|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|Kewujudan Titah A...|{authName -> BERN...|Disember 10, 2024...|                    |[KUALA LUMPUR: Pe...|/berita/nasional/...|
|Kerjasama strateg...|{authName -> , re...|Disember 10, 2024...|                    |[USAHA Jabatan Im...|/berita/nasional/...|
|Terengganu mula d...|{authName -> Wart...|Disember 10, 2024...|       Foto BERNAMA |[KUALA LUMPUR: Te...|/berita/nasional/...|
|RUU Perkongsian D...|{authName -> Suza...|Disember 10, 2024...|Bangunan Parlimen...|[KUALA LUMPUR: Ra...|/berita/nasional/...|
|Imigresen sahkan ...|{authName -> Suza...|Disember 10, 2024...|Ibu Pejabat Suruh...|[KUALA LUMPUR: Ja..

In [3]:
for data in articlesData:
    print(data)

{'title': 'Messi dinobat Pemain Paling Bernilai MLS', 'authDetail': {'authName': '', 'relLink': '', 'email': ''}, 'publishTime': 'Disember 6, 2024 @ 11:49pm', 'articleBrief': 'Messi dinobatkan Pemain Paling Bernilai MLS. Foto REUTERS', 'contents': ['MIAMI: Lionel Messi dinobatkan sebagai Pemain Paling Bernilai (MVP) Major League Soccer (MLS), hari ini selepas memimpin Inter Miami dalam musim yang mencatatkan rekod.', "Bintang Argentina itu menerima anugerah berkenaan hasil sumbangannya yang membawa Miami mencatat jumlah mata musim tertinggi, sekali gus merangkul trofi Supporters' Shield pertama dalam sejarah kelab itu.", 'Messi, 37, menamatkan musim dengan 36 sumbangan gol - paling banyak dalam liga - menerusi 20 jaringan dan 16 umpanan.', 'Messi merangkul anugerah MVP biarpun mengalami musim yang penuh dengan kecederaan yang mana dia hanya membuat 19 penampilan sepanjang tahun ini.', 'Bagaimanapun, musim Miami dan Messi berakhir dengan mengecewakan apabila pasukan itu tersingkir secar

# PERIBAHASA STARTS HERE YA !

In [4]:
peri_scrap = PeriScraper()

proverbs_data_AM = peri_scrap.scrape_page(peri_scrap.AM_Url)
print("Proverbs from A–M:")

for proverb, meaning in proverbs_data_AM:
    print(f"Peribahasa: {proverb} || Maksud: {meaning}")

proverbs_data_NZ = peri_scrap.scrape_page(peri_scrap.NZ_Url)
print("\nProverbs from N–Z:")

for proverb, meaning in proverbs_data_NZ:
    print(f"Peribahasa: {proverb} || Maksud: {meaning}")

24/12/10 22:41:30 WARN Utils: Your hostname, YN. resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/12/10 22:41:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/12/10 22:41:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Processing Table 1...
Processing Table 2...
Processing Table 3...
Processing Table 4...
Processing Table 5...
Processing Table 6...
Processing Table 7...
Processing Table 8...
Processing Table 9...
Processing Table 10...
Processing Table 11...
Processing Table 12...
Processing Table 13...
Processing Table 14...
Proverbs from A–M:
Peribahasa: ada air adalah ikan || Maksud: Ada penempatan adalah penduduk.
Peribahasa: ada aku dipandang hadap tiada aku dipandang belakang || Maksud: Kasih sayang hanya waktu berhadapan saja, setelah berjauhan lalu dilupakan.
Peribahasa: ada angin ada pokoknya pohonnya || Maksud: Segala sesuatu mestilah ada asal mulanya.
Peribahasa: ada bangkai adalah hering || Maksud: Ada perempuan jahat, adalah lelaki jahat yang mengunjunginya.
Peribahasa: ada batang mati adalah cendawan tumbuh || Maksud: Di mana juga kita tinggal, akan ada rezeki kita.
Peribahasa: ada batang cendawan tumbuh || Maksud: Tiap-tiap negeri ada undang-undang dan adat resamnya masing-masing.
Peri